In [1]:
import json

from cbs_net.net import Net
from cbs_net.stochastic import Stochastic
from cbs_net.cargobike import CargoBike
import time

In [2]:
#INPUT DATA

sender_code = 730  # depot location
no_cargobikes = 10

#average package dimensions[mm] for stochastic algorithm
defaultWidth = 100
defaultLength = 100
defaultHeight = 100

In [3]:
n = Net()
print("Loading the network...")
start_time = time.time()
n.load_from_file(fnodes='rynek_nodes.txt', flinks='rynek_links.txt', dlm='\t')
print("Network is loaded in", time.time() - start_time, "sec")
print("Calculating the shortest distances matrix...")
start_time = time.time()
n.sdm = n.floyd_warshall
n.sdm = 0.001 * n.sdm
print("SDM is calculated in", time.time() - start_time, "sec")

Loading the network...
Network is loaded in 0.31148719787597656 sec
Calculating the shortest distances matrix...
SDM is calculated in 341.4232745170593 sec


In [4]:
#add cargobikes
cargobikes = []
capacities = []

for i in range(0, no_cargobikes - 1):
    cargobikes.append(CargoBike())

for singleBike in cargobikes:
    capacities.append(singleBike.capacity)




In [5]:
clients = []
for nd in n.nodes:
    if nd.code < 730:  # 730:
        clients.append(nd)

In [6]:
runs = 1
# 4 rodzaje klientw: sklep, restauracja, bar, inne (hotele, kawiarnie, ...)
probs = {'R': 0.5, 'H': 0.5, 'B': 0.5, 'S': 0.5, 'Ks': 0.5, 'K': 0.5}
weight = 0.03  # average package weight
s_weight = Stochastic(law=1, location=weight, scale=0.25 * weight)


In [7]:
f = open("resultsMP.txt", 'w')
for i in range(runs):
    n.gen_requests(sender=n.get_node(sender_code), nodes=clients,
                   probs=probs, s_weight=s_weight, verbose=False)
    rts = n.genetic(sender_code=sender_code, requests=n.demand,
                    capacity=0.15, verbose=False)
    td, tw = 0, 0
    for rt in rts:
        tw += rt.transport_work
        td += rt.distance
        #print(rt)
    print(i + 1, len(n.demand), td, tw)
    f.write(str(i + 1) + "\t" + str(len(n.demand)) + "\t" + str(td) + "\t" + str(tw) + "\n")
f.close()

1 365 144.59899999999996 11.795458630909875
2 370 152.019 12.396549138733496
3 352 144.57 12.03783790527913
4 375 151.11600000000004 12.330740950566998
5 362 145.198 11.85923663148486


In [8]:
#badanie działania demand
print(len(n.demand))  # demand zawiera listę wszystkich wygenerowanych paczek

demand_dict = vars(n.demand[10])
print(demand_dict)

weight = demand_dict["weight"]

origin_dict = vars(demand_dict['origin'])  # origin jest deklarowany na początku jako sender code
print("Origin: ")
print("Code: " + str(origin_dict["code"]))
print("Name: " + origin_dict["name"])
print("X coord: " + str(origin_dict["x"]))
print("Y coord: " + str(origin_dict["y"]))
print()

destination_dict = vars(demand_dict['destination'])
print(destination_dict)

print("Destination: ")
print("Code: " + str(destination_dict["code"]))
print("Name: " + destination_dict["name"])
print("X coord: " + str(destination_dict["x"]))
print("Y coord: " + str(destination_dict["y"]))
print()
print("Weight: " + str(weight))

print(vars(n.demand[50]))

362
{'weight': 0.03368543594147031, 'origin': <cbs_net.node.Node object at 0x0000018841EB2C08>, 'destination': <cbs_net.node.Node object at 0x0000018841E53B08>, 'width': 0, 'length': 0, 'height': 0, 'm_appear': 0, 'm_load': 0, 'm_unload': 0}
Origin: 
Code: 730
Name: Plac Szczepański 4
X coord: 50.064231
Y coord: 19.93514

{'code': 20, 'name': '1ST Floor Sp. z o.o', 'out_links': [<cbs_net.link.Link object at 0x0000018841EB7FC8>, <cbs_net.link.Link object at 0x0000018841EB7F08>], 'in_links': [<cbs_net.link.Link object at 0x0000018841EB7E88>, <cbs_net.link.Link object at 0x0000018841EBA088>], 's_weight': None, 'req_prob': 1, 'x': 50.060514, 'y': 19.936384, 'type': 'R'}
Destination: 
Code: 20
Name: 1ST Floor Sp. z o.o
X coord: 50.060514
Y coord: 19.936384

Weight: 0.03368543594147031
{'weight': 0.03092581592267576, 'origin': <cbs_net.node.Node object at 0x0000018841EB2C08>, 'destination': <cbs_net.node.Node object at 0x0000018841E61988>, 'width': 0, 'length': 0, 'height': 0, 'm_appear': 0,

In [9]:
#get highest possible dimension, bikes can be heterogenous
#due to solver limitation bikes MUST be homogenous

maxwidth = 0
maxlength = 0
maxheight = 0
for singleBike in cargobikes:

    if maxwidth < singleBike.width:
        maxwidth = singleBike.width

    if maxlength < singleBike.length:
        maxlength = singleBike.length

    if maxheight < singleBike.height:
        maxheight = singleBike.height


In [10]:
# Here package dimensions are checked for fitness
def roll(v): return (v[0], v[2], -v[1])


def turn(v): return (-v[1], v[0], v[2])


def sequence(v):
    for cycle in range(1):
        for step in range(3):  # Yield RTTT 3 times
            v = roll(v)
            yield (v)  #    Yield R
            for i in range(3):  #    Yield TTT
                v = turn(v)
                yield (v)
        v = roll(turn(roll(v)))  # Do RTR


notFitCounter = 0
# add packages dimensions
for singleDemand in n.demand:
    #DEBUG to be deleted
    singleDemand.width = 0
    singleDemand.height = 0
    singleDemand.length = 0
    #DEBUG to be deleted
    while singleDemand.width <= 0:  #ensure that package have positive dimensions
        singleDemand.width = int(Stochastic(law=1, location=defaultWidth, scale=defaultWidth).value())
    while singleDemand.height <= 0:  #ensure that package have positive dimensions
        singleDemand.height = int(Stochastic(law=1, location=defaultHeight, scale=defaultHeight).value())
    while singleDemand.length <= 0:  #ensure that package have positive dimensions
        singleDemand.length = int(Stochastic(law=1, location=defaultHeight, scale=defaultLength).value())

    p = sequence((singleDemand.length, singleDemand.width, singleDemand.height))

    rawRotations = []
    for i in sorted(zip(p)):
        rawRotations.append(i[0])

    rotations = []

    for singleRotation in rawRotations:
        tempTuple = (abs(singleRotation[0]), abs(singleRotation[1]), abs(singleRotation[2]))
        rotations.append(tempTuple)

    rotations = list(set(rotations))

    willFit = 0

    for singleRotation in rotations:  #check if package will fit in cargo bike
        if singleRotation[0] <= maxwidth and singleRotation[1] <= maxlength and singleRotation[2] <= maxheight:
            willFit = 1

    if willFit == 0:
        notFitCounter = notFitCounter + 1

if notFitCounter > 0:
    print(str(notFitCounter) + " packages won't fit.")
    #What to do if package does not fit

# DATA PREPARED FOR SOLVER

In [ ]:

#Starting solver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver.pywrapcp import SolutionCollector

#create routing manager
manager = pywrapcp.RoutingIndexManager(len(n.demand),no_cargobikes, sender_code)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return n.sdm[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

def volume_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    demand_dict = vars(n.demand[from_node])
    volume = demand_dict["length"] * demand_dict["width"] * demand_dict["height"]

    return volume

#volume constraint
volume_callback_index = routing.RegisterUnaryTransitCallback(volume_callback)

routing.AddDimension(
    volume_callback_index,
    0,  # null capacity slack
    (cargobikes[0].length)*(cargobikes[0].height)*(cargobikes[0].width),  # vehicle maximum capacities
    True,  # start cumul to zero
    'Volume')

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

#capacity(weight) constraint
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    demand_dict = vars(n.demand[from_node])
    return demand_dict['weight']

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)




routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    capacities,  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    999999,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Allow to drop nodes.
#penalty = 1000
#for node in range(0, len(n.sdm)-6):
#    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)


 # Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.log_search = True
#search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
#search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#search_parameters.solution_limit = 1
#search_parameters.use_full_propagation = 0
search_parameters.time_limit.FromSeconds(2)

# Solve the problem.

assignment = routing.SolveWithParameters(search_parameters)
collector = initialize_collector(data, manager, routing, distance_matrix)

#search_parameters.solution_limit = 2**24
#search_parameters.time_limit.FromSeconds(200)

routing.SolveFromAssignmentWithParameters(assignment, search_parameters)

routes = []
distances = []

# Print solution on console.
if assignment:

    print('CVRP feasible solutions: {}'.format(collector.SolutionCount()))
    for i in range(collector.SolutionCount()):
        #print(f'================ solution: {i} ================')
        #print_solution(data,orders, manager, routing, collector.Solution(i))
        temp_route,temp_distance = list_solution(data,
                                     manager,
                                     routing,
                                     collector.Solution(i),
                                     i,
                                     n.sdm)

        routes.append(temp_route)
        distances.append(temp_distance)
else:
    print("No solutions")

total_distance = []
for i in distances:
    route_distance = 0
    for j in i:
        vehicle_distance = sum(j)
        route_distance +=vehicle_distance
    total_distance.append(route_distance)




weights = calculate_total_weights(orders,routes)

total_times = []
times = calculate_routes_time(routes, weights, data,orders,distance_matrix)

for solution_time in times:

    total_times.append(sum(solution_time))